<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing a LangChain and Google PaLM Application</h1>

LLM orchestration frameworks such as LangChain provide abstractions that enable users to build powerful applications in a few lines of code. However, the same abstractions can make it difficult to understand what is going on under the hood and to pinpoint the cause of issues.

Phoenix makes your LLM applications *observable* by visualizing the underlying structure of each call to your chain and surfacing problematic "spans" of execution based on latency, token count, or other evaluation metrics.

In this tutorial, you will:
- Build a simple LLM application using LangChain and Google PaLM,
- Record trace data in OpenInference format,
- Inspect the traces and spans of your application to identify uncaught exceptions and sources of latency and cost.

ℹ️️ This notebook requires access to the [Google PaLM API](https://developers.generativeai.google/) and a Google API key.

## 1. Install Dependencies and Import Libraries

Install Phoenix and LangChain.

In [ ]:
!pip install arize-phoenix google-generativeai "langchain<0.2.0"

Import libraries.

In [ ]:
import json
import os
from getpass import getpass
from urllib.request import urlopen

import numpy as np
import pandas as pd
import phoenix as px
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatGooglePalm
from langchain.embeddings import GooglePalmEmbeddings
from langchain.retrievers import KNNRetriever
from phoenix.trace.langchain import LangChainInstrumentor
from tqdm import tqdm

## 2. Launch Phoenix

You can run Phoenix in the background to collect trace data emitted by any LlamaIndex application that has been instrumented with the `OpenInferenceTracer`.

Launch Phoenix and follow the instructions in the cell output to open the Phoenix UI (the UI should be empty because we have yet to run our LangChain application).

In [ ]:
session = px.launch_app()

## 3. Configure Your Google API Key

Set your Google API key.

In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("🔑 Enter your Google API Key: ")

## 4. Instrument LangChain

In [ ]:
LangChainInstrumentor().instrument()

## 5. Build Your LLM Application

Define a `RetrievalQA` chain leveraging "embedding-gecko-001" and "chat-bison-001" from Google PaLM. The knowledge base of this chain is built over the Arize documentation.

In [ ]:
embeddings = GooglePalmEmbeddings(model_name="models/embedding-gecko-001")
database_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/llm/context-retrieval/langchain-vertexai/database.parquet"
)
knn_retriever = KNNRetriever(
    index=np.stack(database_df["text_vector"]),
    texts=database_df["text"].tolist(),
    embeddings=embeddings,
)
llm = ChatGooglePalm(model_name="models/chat-bison-001")
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=knn_retriever,
)

## 6. Run the Chain

Download a small dataset of user queries to ask your application.

In [ ]:
url = "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/llm/context-retrieval/arize_docs_queries.jsonl"
queries = []
with urlopen(url) as response:
    for line in response:
        line = line.decode("utf-8").strip()
        data = json.loads(line)
        queries.append(data["query"])
queries[:10]

Run your chain against ten queries.

In [ ]:
for query in tqdm(queries[:10]):
    chain.invoke(query)

Check out Phoenix to view your collected spans and traces!

In [ ]:
print(f"Open the Phoenix UI if you haven't already: {session.url}")

## 7. Export Your Trace Data

You can export your trace data as a pandas dataframe for further analysis and evaluation.

In this case, we will export our retriever spans and view them in a pandas dataframe.

In [ ]:
trace_df = px.Client().get_spans_dataframe('span_kind == "RETRIEVER"')
trace_df